In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import random

# Configurable CNN Model
class CNN(nn.Module):
    def __init__(self, activation='relu', init='xavier'):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)  # CIFAR-10 has 10 classes
        self.dropout = nn.Dropout(0.5)
        self.batch_norm = nn.BatchNorm2d(128)
        
        self.activation = self.get_activation(activation)
        self.init_weights(init)

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = self.pool(self.activation(self.conv3(x)))
        x = self.batch_norm(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.activation(self.fc1(x)))
        x = self.fc2(x)
        return x
    
    def get_activation(self, name):
        if name == 'relu':
            return F.relu
        elif name == 'tanh':
            return F.tanh
        elif name == 'leaky_relu':
            return F.leaky_relu
    
    def init_weights(self, init_type):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                if init_type == 'xavier':
                    nn.init.xavier_uniform_(m.weight)
                elif init_type == 'kaiming':
                    nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif init_type == 'random':
                    nn.init.uniform_(m.weight, -0.1, 0.1)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

# Load CIFAR-10 Dataset
def get_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# Training Function
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    return model

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Configurations to Test
activations = ['relu', 'tanh', 'leaky_relu']
initializations = ['xavier', 'kaiming', 'random']
optimizers = {'sgd': optim.SGD, 'adam': optim.Adam, 'rmsprop': optim.RMSprop}

# Run Experiments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader = get_dataloader()

best_accuracy = 0
best_model = None

for activation in activations:
    for init in initializations:
        for opt_name, opt_fn in optimizers.items():
            print(f"Training CNN with {activation}, {init}, {opt_name}")
            model = CNN(activation, init).to(device)
            optimizer = opt_fn(model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()
            model = train_model(model, train_loader, optimizer, criterion, epochs=10)
            acc = evaluate_model(model, test_loader)
            if acc > best_accuracy:
                best_accuracy = acc
                best_model = model

# Save the Best Model
torch.save(best_model.state_dict(), "best_cnn_model.pth")
print("Best model saved with accuracy:", best_accuracy)


100%|██████████| 170M/170M [00:01<00:00, 94.1MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training CNN with relu, xavier, sgd
Epoch [1/10], Loss: 2.1215
Epoch [2/10], Loss: 1.7486
Epoch [3/10], Loss: 1.6228
Epoch [4/10], Loss: 1.5421
Epoch [5/10], Loss: 1.4740
Epoch [6/10], Loss: 1.4214
Epoch [7/10], Loss: 1.3769
Epoch [8/10], Loss: 1.3410
Epoch [9/10], Loss: 1.3092
Epoch [10/10], Loss: 1.2761
Test Accuracy: 59.00%
Training CNN with relu, xavier, adam
Epoch [1/10], Loss: 1.4567
Epoch [2/10], Loss: 1.0385
Epoch [3/10], Loss: 0.8670
Epoch [4/10], Loss: 0.7542
Epoch [5/10], Loss: 0.6656
Epoch [6/10], Loss: 0.5969
Epoch [7/10], Loss: 0.5388
Epoch [8/10], Loss: 0.4865
Epoch [9/10], Loss: 0.4367
Epoch [10/10], Loss: 0.3953
Test Accuracy: 77.46%
Training CNN with relu, xavier, rmsprop
Epoch [1/10], Loss: 1.5602
Epoch [2/10], Loss: 1.1025
Epoch [3/10], Loss: 0.9285
Epoch [4/10], Loss: 0.8042
Epoch [5/10], Loss: 0.7198
Epoch [6/10], Loss: 0.6478
Epoch [7/10], Loss: 0.5823
Epoch [8/10], Loss: 0.5

# **Fine Tuned Resnet-18**

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn.functional as F

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fine-Tune ResNet-18 for CIFAR-10 with Extra Layers
def fine_tune_resnet18():
    model = models.resnet18(pretrained=True)

    # Unfreeze the last 2 blocks for fine-tuning
    for param in model.parameters():
        param.requires_grad = False  # Freeze all layers

    for param in list(model.layer4.parameters()):  # Unfreeze the last block
        param.requires_grad = True

    for param in list(model.layer3.parameters()):  # Unfreeze the second last block
        param.requires_grad = True

    num_ftrs = model.fc.in_features

    # Modify the classifier with extra layers
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 1024),  
        nn.LeakyReLU(negative_slope=0.01),
        nn.BatchNorm1d(1024),
        nn.Linear(1024, 512),
        nn.LeakyReLU(negative_slope=0.01),
        nn.BatchNorm1d(512),
        nn.Dropout(0.3),
        nn.Linear(512, 10)  # Output layer for CIFAR-10 (10 classes)
    )
    
    return model.to(device)

# Load CIFAR-10 dataset
def get_cifar10_dataloader(batch_size=64):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to match ResNet input size
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Training Function
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    return model

# Evaluation Function
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Load Data
train_loader, test_loader = get_cifar10_dataloader(batch_size=64)

# Configurations for Optimizer and Loss Function
optimizers = {'sgd': optim.SGD, 'adam': optim.Adam, 'rmsprop': optim.RMSprop}
criterion = nn.CrossEntropyLoss()

# Fine-tune ResNet-18
best_accuracy = 0
best_model = None

for opt_name, opt_fn in optimizers.items():
    print(f"Training ResNet-18 with optimizer: {opt_name}")
    model = fine_tune_resnet18()
    optimizer = opt_fn(model.parameters(), lr=0.001)
    model = train_model(model, train_loader, optimizer, criterion, epochs=10)
    acc = evaluate_model(model, test_loader)
    
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model

# Save the Best Model
torch.save(best_model.state_dict(), "best_resnet18_model.pth")
print("Best model saved with accuracy:", best_accuracy)


Files already downloaded and verified
Files already downloaded and verified
Training ResNet-18 with optimizer: sgd
Epoch [1/10], Loss: 1.0600
Epoch [2/10], Loss: 0.5123
Epoch [3/10], Loss: 0.4050
Epoch [4/10], Loss: 0.3458
Epoch [5/10], Loss: 0.3066
Epoch [6/10], Loss: 0.2746
Epoch [7/10], Loss: 0.2481
Epoch [8/10], Loss: 0.2307
Epoch [9/10], Loss: 0.2069
Epoch [10/10], Loss: 0.1893
Test Accuracy: 90.79%
Training ResNet-18 with optimizer: adam
Epoch [1/10], Loss: 0.5173
Epoch [2/10], Loss: 0.2836
Epoch [3/10], Loss: 0.1976
Epoch [4/10], Loss: 0.1371
Epoch [5/10], Loss: 0.1005
Epoch [6/10], Loss: 0.0844
Epoch [7/10], Loss: 0.0684
Epoch [8/10], Loss: 0.0576
Epoch [9/10], Loss: 0.0557
Epoch [10/10], Loss: 0.0495
Test Accuracy: 89.87%
Training ResNet-18 with optimizer: rmsprop
Epoch [1/10], Loss: 0.6838
Epoch [2/10], Loss: 0.3639
Epoch [3/10], Loss: 0.2251
Epoch [4/10], Loss: 0.1423
Epoch [5/10], Loss: 0.1072
Epoch [6/10], Loss: 0.0791
Epoch [7/10], Loss: 0.0654
Epoch [8/10], Loss: 0.0560
